In [1]:
local = True

In [2]:
if not local:
    from google.colab import drive
    drive.mount('/content/drive')#, force_remount=True)

In [3]:
if not local:
    %cd /content/drive/MyDrive/Sun/ML Shock/Final project/

# Convolutional Neural Networks


In [4]:
import sys
if local:
    sys.path.insert(0,"C:/Users/Amy/Desktop/Green_Git/eegClassification/utils")
else:
    sys.path.insert(0,"/content/drive/MyDrive/Sun/ML Shock/Final project/")

In [5]:
# Import libraries
import torch
import numpy as np
from tqdm import tqdm

from CustomDataLoaderNPY import CustomDataset
from torch.utils.data import DataLoader
from model_architectures import CustomCNN, TransNet_Resnet18, TransNet_Efficientnetb0

import os

In [6]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    print("GPU count", torch.cuda.device_count())

CUDA is not available.  Training on CPU ...


---
## Load the Data

In [7]:
# Load (train or test) data from csv file
if local:
    path = "C:/Users/Amy/Desktop/Green_Git/eegClassification/data/"
    path_df = "C:/Users/Amy/Desktop/Green_Git/eegClassification/files/"
else:
    path = "./data_prep_all_spec/"
    path_df = "./data/"


In [8]:
# Type of input data
data_type = "eeg_spec"  #"spec" # "eeg_raw" #
# number of subprocesses to use for data loading
import multiprocessing as cpu
num_workers = 0 #cpu.cpu_count() #- 1
# how many samples per batch to load
batch_size = 64
# label smoothing
label_smoothing = 0.01

In [9]:
if data_type == "spec":
    input_shape = (3,400,299)
elif data_type == "eeg_spec":
    input_shape = (20,129,43)
elif data_type == "eeg_raw":
    input_shape = (20,9800)

In [10]:
data_path = path+f'data_prep_all_{data_type}/train/'
data_files = os.listdir(data_path)
data_files = [f.split('.')[0].split('_')[1] for f in data_files if f.startswith('images')]
N_items = len(data_files)
text = 'features'

print("Number of items", N_items)

Number of items 700


In [11]:
# specify the image classes
classes = [
        "seizure_vote",
        "lpd_vote",
        "gpd_vote",
        "lrda_vote",
        "grda_vote",
        "other_vote",
    ]
N_classes = len(classes)

In [12]:
get_batch_transform = lambda x, y: (
    x[0, :],
    y[0, :],
)


## Define the Network Architecture


In [13]:
model_name = "TransNet_Resnet18" #"CustomCNN" # "TransNet_Efficientnetb0" #


if model_name == "CustomCNN":
  model = CustomCNN(input_shape=input_shape, N_classes = N_classes)
elif model_name == "TransNet_Resnet18":
  model = TransNet_Resnet18(input_shape=input_shape, N_classes = N_classes)
elif model_name == "TransNet_Efficientnetb0":
  model = TransNet_Efficientnetb0(input_shape=input_shape, N_classes = N_classes)
else:
  raise ValueError("Model not found")



c:\Users\Amy\Desktop\Green_Git\eegClassification\.venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Amy\Desktop\Green_Git\eegClassification\.venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Amy/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:15<00:00, 2.96MB/s]


In [14]:
# index = 0
# path_model_old = f"./model_{model_name}_{index}.pt"
# model.load_state_dict(torch.load(path_model_old))


In [15]:
model

TransNet_Resnet18(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, t

In [16]:
# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

In [17]:
num_parameters = sum(p.numel() for p in model.parameters())
print('Number of parameters in the model', num_parameters)

# number of trainable parameters
num_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Number of trainable parameters in the model', num_parameters)

Number of parameters in the model 11176512
Number of trainable parameters in the model 0


In [18]:
path_out = f"./{model_name}_{text}_{data_type}/"

if not os.path.exists(path_out):
    os.makedirs(path_out)


loader = DataLoader(CustomDataset(data_path, data_files), batch_size=1, shuffle=False, num_workers=num_workers)

count = 0
for data, votes in tqdm(loader):

    data, votes = get_batch_transform(data, votes)
    # votes = votes * (1 - label_smoothing) + label_smoothing / N_classes

    if train_on_gpu:
        data, votes = data.cuda(), votes.cuda()

    output = model(data)

    # save output as npy
    output = output.cpu().detach().numpy()
    np.save(path_out + f"{text}_{count}.npy", output)

    votes = votes.cpu().detach().numpy()
    np.save(path_out + f"votes_{count}.npy", votes)

    count += 1

  0%|          | 0/700 [00:00<?, ?it/s]

100%|██████████| 700/700 [44:37<00:00,  3.82s/it]  


# Reload inferred data

In [ ]:
text = 'features'
data_path = path+f'data/{model_name}_{text}_{data_type}/'
data_files = os.listdir(data_path)
data_features = [f for f in data_files if f.startswith(text)]
data_votes = [f for f in data_files if f.startswith('votes')]
N_items = len(data_features)

print("Number of items", N_items)

In [ ]:

np.load(data_path+data_features[0]).shape, np.load(data_path+data_votes[0]).shape

In [ ]:
# load all data_features into a single numpy array
X = np.zeros((N_items,*input_shape))
for i in range(N_items):
    X[i] = np.load(data_path+data_features[i])

# load all data_votes into a single numpy array
Y = np.zeros((N_items, batch_size, N_classes))
for i in range(N_items):
    Y[i] = np.load(data_path+data_votes[i])

In [ ]:
X.shape, Y.shape

In [ ]:
X = X.reshape(N_items*batch_size, N_classes)
Y = Y.reshape(N_items*batch_size, N_classes)

In [ ]:
X.shape, Y.shape

In [ ]:
# convert votes to class probabilities
Y = Y/np.sum(Y, axis=1)[:,np.newaxis]
X = X/np.sum(X, axis=1)[:,np.newaxis]

In [ ]:
predictions = np.argmax(X, axis=1)
y_test_class = np.argmax(Y, axis=1)

In [ ]:
predictions[0], y_test_class[0]

In [ ]:
# evaluate predictions
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test_class, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

confusion_matrix = confusion_matrix(y_test_class, predictions)

plt.figure(figsize=(10, 10))
sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap='Blues')
plt.xticks(ticks=np.arange(6) + 0.5, labels=classes)
plt.yticks(ticks=np.arange(6) + 0.5, labels=classes)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
# evaluate using kl divergence
from scipy.stats import entropy
kl_div = np.mean([entropy(Y[i], X[i]) for i in range(len(Y))])
lk_base = np.mean([entropy(Y[i], np.array([1/6]*6)) for i in range(len(Y))])
print(f"KL div: {kl_div:.3f} (baseline: {lk_base:.3f})")